### Creating and Persisting an ML Model

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [ ]:
df.describe()
df

In [ ]:
df.info

Create a subset of features as an example.

In [ ]:
include = ['school', 'higher','Mjob', 'Fjob', 'studytime', 'paid', 'failures', 'G3']
df.drop(columns=df.columns.difference(include), inplace=True)  # only using 3 features

In [ ]:
df.info

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [ ]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [ ]:
df.describe()

Drop the G3 score

In [ ]:
include = ['school', 'higher','Mjob', 'Fjob', 'studytime', 'paid', 'failures', 'qual_student']
df.drop(columns=df.columns.difference(include), inplace=True) 

Import scikit-learn and build a random forest classifer

In [ ]:
import sklearn
from sklearn import preprocessing, neighbors, svm
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import train_test_split

dependent_variable = 'qual_student'

df = pd.get_dummies(df, drop_first=True)
# df = pd.concat(pd.get_dummies(df, drop_first=True), df,  axis=1)
# df = pd.get_dummies(df, drop_first=True).astype(float)
y = df[dependent_variable]
X = df[df.columns.difference([dependent_variable])]
# print(X)

# print(df)

# clf = rf(n_estimators = 1000)
# clf.fit(x, y)

#splitting the train and test sets
X_train, X_test, y_train,y_test= train_test_split(X, y, test_size=0.3)

In [ ]:
# pred = clf.predict(x)
# sklearn.metrics.f1_score(y, pred, average='binary')

rfc = rf(criterion='gini', 
        n_estimators=10750,
        max_depth=5,
        min_samples_split=6,
        min_samples_leaf=6,
        max_features='auto',
        oob_score=True,
        random_state=42,
        n_jobs=-1,
        verbose=1)

rfc.fit(X_train, y_train)
print('****Results****')
print("Accuracy: {:.4%}".format(rfc.score(X_test, y_test)))

It's not very good! We didn't even cross validate. You'll need to do better :)
Let's export this model so we can use it in a microservice (flask api)

In [ ]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(rfc, 'app/handlers/model4.pkl')

In [ ]:
# print(X_train)
# print(X_train.columns)
query_df = pd.DataFrame({'Fjob_health' : pd.Series(0), 
                         'Fjob_other' : pd.Series(1), 
                         'Fjob_services': pd.Series(0),
                         'Fjob_teacher': pd.Series(0),
                         'Mjob_health': pd.Series(0),
                         'Mjob_other': pd.Series(0),
                         'Mjob_services': pd.Series(0),
                         'Mjob_teacher': pd.Series(1),
                         'failures': pd.Series(5),
                         'higher_yes': pd.Series(1),
                         'paid_yes': pd.Series(1),
                         'school_MS': pd.Series(0),
                         'studytime': pd.Series(20)})
# query_df = pd.get_dummies(query_df, drop_first=True).astype(float)

In [ ]:
pred = rfc.predict(query_df)

In [ ]:
pred

In [ ]:
type(pred)

In [ ]:
print(query_df.columns)